In [1]:
from tenacity import (  # for exponential backoff
    retry,
    stop_after_attempt,
    wait_random_exponential,
)
import openai

class OpenAIAPI:
    def __init__(self, api_key,
                 model="gpt-4-0613",
                 api_base="http://localhost:9581/v1"
                 ):

        self.api_key = api_key
        self.model = model
        self.api_base = api_base

    def chat_completion(self, messages, temperature=0.7, **kwargs):
        return self.chat_completion_with_backoff(messages=messages,
                                                 temperature=temperature,
                                                 **kwargs)

    @retry(wait=wait_random_exponential(min=1, max=10), stop=stop_after_attempt(50), reraise=False)
    def chat_completion_with_backoff(self, messages, temperature, **kwargs):
        if "gpt" in self.model:
            if openai.__version__ > '1.0.0':
                client = openai.AzureOpenAI(
                    azure_endpoint=self.api_base,
                    api_version="2023-07-01-preview",
                    api_key=self.api_key
                )
                return client.chat.completions.create(
                    model=self.model,
                    messages=messages,
                    temperature=temperature,
                    **kwargs
                )
            else:
                openai.api_key = self.api_key
                openai.api_base = self.api_base
                openai.api_type = "azure"
                openai.api_version = "2023-07-01-preview"
                return openai.ChatCompletion.create(
                    engine=self.model,
                    messages=messages,
                    temperature=temperature,
                    **kwargs
                )

        else:
            openai.api_key = 'none'
            openai.api_base = self.api_base
            return openai.ChatCompletion.create(
                model=self.model,
                messages=messages,
                temperature=temperature,
                **kwargs
            )


In [ ]:
gpt4 = OpenAIAPI(model="gpt-4-1106-preview",
                     api_key="O4ga4drYc3lOFnHYq6HDmQkvCFjWNgyW",
                     api_base="http://0.0.0.0:4400",
                     )

response = gpt4.chat_completion(messages=[{"role": "user", "content": "test"}],
                                    temperature=0.7)
print(response.choices[0].message.content)

In [3]:
from py2neo import Graph

# Neo4j 数据库连接信息
uri = "bolt://localhost:7688"
username = "neo4j"
password = "ainet666"

# 连接到数据库
graph = Graph(uri, auth=(username, password))

In [ ]:
# 初始化数据库内容
init_script = """
MERGE (m:Movie {name:"Top Gun"})
WITH m
UNWIND ["Tom Cruise", "Val Kilmer", "Anthony Edwards", "Meg Ryan"] AS actor
MERGE (a:Actor {name:actor})
MERGE (a)-[:ACTED_IN]->(m)
"""
graph.run(init_script)

In [ ]:
# 传给GPT的database schema
db_schema = ""
results = graph.run("""
call db.schema.nodeTypeProperties
""")
for record in results:
    db_schema += str(record) + "\n"
    
results = graph.run("""
call db.schema.relTypeProperties
""")
for record in results:
    db_schema += str(record) + "\n"
print("db_schema is: ", db_schema)

In [38]:
question = "How many people played in Top Gun?"

response = gpt4.chat_completion(messages=[
        {"role": "system", "content": "Generate Cypher script for the following question to query a neo4j database, and please reponse the script which I can directly run, without any other words or '```cypher'."},
        {"role": "system", "content": "The schema of Noe4j database is" + db_schema},
        {"role": "system", "content": "Please use the schema above to generate your answer."},
        {"role": "user", "content": question},
    ], temperature=0.7)
gpt_script = response.choices[0].message.content
print("script generated: ", gpt_script, '\n')

results = graph.run(gpt_script)
for record in results:
    print(record)
    


script generated:  MATCH (:Movie {name: "Top Gun"})<-[:ACTED_IN]-(a:Actor)
RETURN count(a) as NumberOfActors 

4


In [8]:
# 清空所有数据
graph.run("""
    MATCH (n)
    DETACH DELETE n
""")

(No data)